In [1]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, ForeignKey

In [2]:
engine = create_engine("sqlite:///prueba.sqlite", echo=True)

In [3]:
Base = declarative_base()

In [4]:
class User(Base):
    __tablename__ = "users"
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    nickname = Column(String)
    
    def __repr__(self):
        return f"<User(name='{self.name}', fullname='{self.fullname}', nickname='{self.nickname}')>"

In [5]:
User.__table__

Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), Column('nickname', String(), table=<users>), schema=None)

In [6]:
Base.metadata.create_all(engine)

2019-12-08 19:19:48,573 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-12-08 19:19:48,576 INFO sqlalchemy.engine.base.Engine ()
2019-12-08 19:19:48,578 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-12-08 19:19:48,580 INFO sqlalchemy.engine.base.Engine ()
2019-12-08 19:19:48,583 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("users")
2019-12-08 19:19:48,585 INFO sqlalchemy.engine.base.Engine ()
2019-12-08 19:19:48,587 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("users")
2019-12-08 19:19:48,589 INFO sqlalchemy.engine.base.Engine ()
2019-12-08 19:19:48,602 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	nickname VARCHAR, 
	PRIMARY KEY (id)
)


2019-12-08 19:19:48,610 INFO sqlalchemy.engine.base.Engine ()
2019-12-08 19:19:48,618 INFO sqlalchemy.engine.base.Engine COMMIT


In [7]:
usuario1 = User(name="roberto", fullname="Roberto Cadena Vega", nickname="rob")

In [8]:
from sqlalchemy.orm import sessionmaker

In [9]:
sesion = sessionmaker(bind=engine)

In [10]:
ses1 = sesion()

In [11]:
ses1.add(usuario1)

In [12]:
datos = ses1.query(User).filter_by(name='roberto').first() 

2019-12-08 19:20:34,319 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-12-08 19:20:34,321 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2019-12-08 19:20:34,322 INFO sqlalchemy.engine.base.Engine ('roberto', 'Roberto Cadena Vega', 'rob')
2019-12-08 19:20:34,327 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2019-12-08 19:20:34,332 INFO sqlalchemy.engine.base.Engine ('roberto', 1, 0)


In [13]:
datos

<User(name='roberto', fullname='Roberto Cadena Vega', nickname='rob')>

In [14]:
datos == usuario1

True

In [15]:
ses1.commit()

2019-12-08 19:20:39,670 INFO sqlalchemy.engine.base.Engine COMMIT


In [16]:
usuarios_nuevos = [{"name":"leticia" , "fullname":"Maria Leticia Vega Quiroz" , "nickname":"lety"},
                   {"name":"francisco" , "fullname":"Francisco Cadena Cruz" , "nickname":"paco"},
                   {"name":"alberto" , "fullname":"Alberto Cadena Vega" , "nickname":"bebe"}]

In [17]:
for usuario in usuarios_nuevos:
    ses1.add(User(name=usuario["name"], fullname=usuario["fullname"], nickname=usuario["nickname"]))

In [18]:
ses1.commit()

2019-12-08 19:20:50,677 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-12-08 19:20:50,680 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2019-12-08 19:20:50,682 INFO sqlalchemy.engine.base.Engine ('leticia', 'Maria Leticia Vega Quiroz', 'lety')
2019-12-08 19:20:50,684 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2019-12-08 19:20:50,688 INFO sqlalchemy.engine.base.Engine ('francisco', 'Francisco Cadena Cruz', 'paco')
2019-12-08 19:20:50,690 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2019-12-08 19:20:50,691 INFO sqlalchemy.engine.base.Engine ('alberto', 'Alberto Cadena Vega', 'bebe')
2019-12-08 19:20:50,693 INFO sqlalchemy.engine.base.Engine COMMIT


In [19]:
class Event(Base):
    __tablename__ = "events"
    
    id = Column(Integer, primary_key=True)
    user = Column(Integer, ForeignKey('users.id'))
    date = Column(String)
    event = Column(String)
    
    def __repr__(self):
        return f"<Event(date='{self.date}', event='{self.event}')>"

In [20]:
Base.metadata.create_all(engine)

2019-12-08 19:22:13,664 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("users")
2019-12-08 19:22:13,668 INFO sqlalchemy.engine.base.Engine ()
2019-12-08 19:22:13,675 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("events")
2019-12-08 19:22:13,677 INFO sqlalchemy.engine.base.Engine ()
2019-12-08 19:22:13,679 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("events")
2019-12-08 19:22:13,680 INFO sqlalchemy.engine.base.Engine ()
2019-12-08 19:22:13,700 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE events (
	id INTEGER NOT NULL, 
	user INTEGER, 
	date VARCHAR, 
	event VARCHAR, 
	PRIMARY KEY (id), 
	FOREIGN KEY(user) REFERENCES users (id)
)


2019-12-08 19:22:13,704 INFO sqlalchemy.engine.base.Engine ()
2019-12-08 19:22:13,713 INFO sqlalchemy.engine.base.Engine COMMIT


In [21]:
def logger(sesion, nom_usuario, evento):
    from datetime import datetime
    datos = sesion.query(User).filter_by(name=nom_usuario).all()
    if len(datos) == 1:
        evento1 = Event(user=datos[0].id, date=datetime.now().strftime("%y/%m/%d %H:%M"), event=evento)
        sesion.add(evento1)
        sesion.commit()
    else:
        print("El usuario no pertenece al sistema.")

In [22]:
logger(ses1, "roberto", "desayuno")

2019-12-08 19:22:18,963 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-12-08 19:22:18,967 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?
2019-12-08 19:22:18,969 INFO sqlalchemy.engine.base.Engine ('roberto',)
2019-12-08 19:22:18,978 INFO sqlalchemy.engine.base.Engine INSERT INTO events (user, date, event) VALUES (?, ?, ?)
2019-12-08 19:22:18,982 INFO sqlalchemy.engine.base.Engine (1, '19/12/08 19:22', 'desayuno')
2019-12-08 19:22:18,989 INFO sqlalchemy.engine.base.Engine COMMIT


In [23]:
logger(ses1, "francisco", "Salida a YMCA")

2019-12-08 19:22:19,814 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-12-08 19:22:19,817 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?
2019-12-08 19:22:19,819 INFO sqlalchemy.engine.base.Engine ('francisco',)
2019-12-08 19:22:19,822 INFO sqlalchemy.engine.base.Engine INSERT INTO events (user, date, event) VALUES (?, ?, ?)
2019-12-08 19:22:19,824 INFO sqlalchemy.engine.base.Engine (3, '19/12/08 19:22', 'Salida a YMCA')
2019-12-08 19:22:19,827 INFO sqlalchemy.engine.base.Engine COMMIT


In [24]:
def tabla_eventos(sesion):
    for evento in sesion.query(Event).order_by(Event.date)[-5:]:
        print(evento)

In [25]:
tabla_eventos(ses1)

2019-12-08 19:22:29,987 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-12-08 19:22:29,990 INFO sqlalchemy.engine.base.Engine SELECT events.id AS events_id, events.user AS events_user, events.date AS events_date, events.event AS events_event 
FROM events ORDER BY events.date
2019-12-08 19:22:29,992 INFO sqlalchemy.engine.base.Engine ()
<Event(date='19/12/08 19:22', event='desayuno')>
<Event(date='19/12/08 19:22', event='Salida a YMCA')>


In [26]:
ses1.query(Event).order_by(Event.date).scalar()

2019-12-08 19:22:31,299 INFO sqlalchemy.engine.base.Engine SELECT events.id AS events_id, events.user AS events_user, events.date AS events_date, events.event AS events_event 
FROM events ORDER BY events.date
2019-12-08 19:22:31,301 INFO sqlalchemy.engine.base.Engine ()


MultipleResultsFound: Multiple rows were found for one()

In [27]:
logger(ses1, "francisco", "partido del america")

2019-12-08 19:22:34,620 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?
2019-12-08 19:22:34,623 INFO sqlalchemy.engine.base.Engine ('francisco',)
2019-12-08 19:22:34,626 INFO sqlalchemy.engine.base.Engine INSERT INTO events (user, date, event) VALUES (?, ?, ?)
2019-12-08 19:22:34,627 INFO sqlalchemy.engine.base.Engine (3, '19/12/08 19:22', 'partido del america')
2019-12-08 19:22:34,631 INFO sqlalchemy.engine.base.Engine COMMIT


In [28]:
logger(ses1, "roberto", "snacks")

2019-12-08 19:22:35,452 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-12-08 19:22:35,456 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?
2019-12-08 19:22:35,458 INFO sqlalchemy.engine.base.Engine ('roberto',)
2019-12-08 19:22:35,463 INFO sqlalchemy.engine.base.Engine INSERT INTO events (user, date, event) VALUES (?, ?, ?)
2019-12-08 19:22:35,465 INFO sqlalchemy.engine.base.Engine (1, '19/12/08 19:22', 'snacks')
2019-12-08 19:22:35,468 INFO sqlalchemy.engine.base.Engine COMMIT


In [29]:
logger(ses1, "leticia", "partido del monarcas")

2019-12-08 19:22:36,105 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-12-08 19:22:36,108 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?
2019-12-08 19:22:36,110 INFO sqlalchemy.engine.base.Engine ('leticia',)
2019-12-08 19:22:36,113 INFO sqlalchemy.engine.base.Engine INSERT INTO events (user, date, event) VALUES (?, ?, ?)
2019-12-08 19:22:36,115 INFO sqlalchemy.engine.base.Engine (2, '19/12/08 19:22', 'partido del monarcas')
2019-12-08 19:22:36,118 INFO sqlalchemy.engine.base.Engine COMMIT
